<a href="https://colab.research.google.com/github/Zee3615/test/blob/master/Weed_Recommend_d2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# data set:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
from sqlalchemy import create_engine
import warnings
import seaborn as sns
from scipy.stats.mstats import winsorize
from statistics import mode
import scipy

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


DATA

In [ ]:
import pandas as pd
df=pd.read_csv('gdrive/My Drive/Weed_Recommender/data/strains_9000.csv')

In [ ]:
#display full content of a column 
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [ ]:
pd.set_option('display.max_row', None)

In [ ]:
df.head(10)

,Unnamed: 0,name,effects,ailment,flavor,location,terpenes,thc,thca,thcv,cbd,cbda,cbdv,cbn,cbg,cbgm,cbgv,cbc,cbcv,cbv,cbt,cbl
0,0,100 OG,Focused,Depression,Citrus,NaN,Limonene,127,0.0,0.0,16,0.0,0.0,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,707 Headband,NaN,NaN,NaN,NaN,NaN,127,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,A-10,"Relaxed, Happy, Uplifted, Energetic, Sleepy, Dry Mouth, Paranoid","Stress, Insomnia, Pain, Muscle Spasms, Depression","Citrus, Sweet",NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Acapulco Gold,"Happy, Euphoric, Uplifted, Relaxed, Creative, Dry Mouth, Paranoid, Anxious","Depression, Stress, Pain, Lack of Appetite","Earthy, Citrus",NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Afghani Bullrider,"Uplifted, Relaxed, Happy, Euphoric, Dry Mouth, Paranoid","Stress, Depression, Insomnia, Pain","Sweet, Pine, Earthy",NaN,NaN,127,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,Afghan Big Bud,"Euphoric, Happy, Talkative, Relaxed, Sleepy, Dry Mouth, Paranoid","Stress, Pain, Nausea, Insomnia, Depression","Lemon, Lavender",NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,Afghan Diesel,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,Afghan Kush,NaN,NaN,NaN,"Yucca Valley, CA",NaN,1300,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,Afghani,"Relaxed, Sleepy, Euphoric, Happy, Uplifted, Dry Mouth, Paranoid","Stress, Insomnia, Pain, Lack of Appetite, Depression","Earthy, Sweet, Pine",NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,Afghanica,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9524 entries, 0 to 9523
Data columns (total 22 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  9524 non-null   int64  
 1   name        9523 non-null   object 
 2   effects     1015 non-null   object 
 3   ailment     971 non-null    object 
 4   flavor      971 non-null    object 
 5   location    621 non-null    object 
 6   terpenes    1 non-null      object 
 7   thc         9524 non-null   int64  
 8   thca        8635 non-null   float64
 9   thcv        8635 non-null   float64
 10  cbd         9524 non-null   int64  
 11  cbda        8635 non-null   float64
 12  cbdv        8635 non-null   float64
 13  cbn         9524 non-null   int64  
 14  cbg         8635 non-null   float64
 15  cbgm        8635 non-null   float64
 16  cbgv        8635 non-null   float64
 17  cbc         8635 non-null   float64
 18  cbcv        8635 non-null   float64
 19  cbv         8635 non-null  

In [ ]:
df.describe()

,Unnamed: 0,thc,thca,thcv,cbd,cbda,cbdv,cbn,cbg,cbgm,cbgv,cbc,cbcv,cbv,cbt,cbl
count,9524.000000,9524.000000,8635.0,8635.0,9524.000000,8635.0,8635.0,9524.000000,8635.0,8635.0,8635.0,8635.0,8635.0,8635.0,8635.0,8635.0
mean,4761.500000,106.347648,0.0,0.0,3.749895,0.0,0.0,364.421252,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,2749.486316,421.578887,0.0,0.0,62.131418,0.0,0.0,6209.909450,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,2380.750000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,4761.500000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,7142.250000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,9523.000000,7700.000000,0.0,0.0,1900.000000,0.0,0.0,190000.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#convert object to string
col_1 = ['effects','ailment','flavor']

for column in col_1:
  df[column] = df[column].apply(lambda x: str(x))
  #arrange alphabetically
  df[column] = [','.join(sorted(x.split(','))) for x in df[column]]

In [ ]:
df.head(10)

,Unnamed: 0,name,effects,ailment,flavor,location,terpenes,thc,thca,thcv,cbd,cbda,cbdv,cbn,cbg,cbgm,cbgv,cbc,cbcv,cbv,cbt,cbl
0,0,100 OG,Focused,Depression,Citrus,NaN,Limonene,127,0.0,0.0,16,0.0,0.0,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,707 Headband,nan,nan,nan,NaN,NaN,127,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,A-10,"Dry Mouth, Energetic, Happy, Paranoid, Sleepy, Uplifted,Relaxed","Depression, Insomnia, Muscle Spasms, Pain,Stress","Sweet,Citrus",NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Acapulco Gold,"Anxious, Creative, Dry Mouth, Euphoric, Paranoid, Relaxed, Uplifted,Happy","Lack of Appetite, Pain, Stress,Depression","Citrus,Earthy",NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Afghani Bullrider,"Dry Mouth, Euphoric, Happy, Paranoid, Relaxed,Uplifted","Depression, Insomnia, Pain,Stress","Earthy, Pine,Sweet",NaN,NaN,127,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,Afghan Big Bud,"Dry Mouth, Happy, Paranoid, Relaxed, Sleepy, Talkative,Euphoric","Depression, Insomnia, Nausea, Pain,Stress","Lavender,Lemon",NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,Afghan Diesel,nan,nan,nan,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,Afghan Kush,nan,nan,nan,"Yucca Valley, CA",NaN,1300,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,Afghani,"Dry Mouth, Euphoric, Happy, Paranoid, Sleepy, Uplifted,Relaxed","Depression, Insomnia, Lack of Appetite, Pain,Stress","Pine, Sweet,Earthy",NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,Afghanica,nan,nan,nan,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
highest = list(sorted(df['effects'].str.len(), reverse=True))[0]
highest

df.loc[df['effects'].str.len() == highest]

,Unnamed: 0,name,effects,ailment,flavor,location,terpenes,thc,thca,thcv,cbd,cbda,cbdv,cbn,cbg,cbgm,cbgv,cbc,cbcv,cbv,cbt,cbl
189,189,J-27,"Anxious, Creative, Dry Mouth, Euphoric, Focused, Paranoid, Uplifted,Energetic","Nausea, Pain,Stress","Orange,Citrus",NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1693,1693,White Durban,"Anxious, Creative, Dry Mouth, Euphoric, Focused, Paranoid, Uplifted,Energetic","Depression, Pain,Stress",Earthy,NaN,NaN,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
import re

for column in col_1:
  df[column] = df[column].str.replace(r" ","")

df.head(5)

,Unnamed: 0,name,effects,ailment,flavor,location,terpenes,thc,thca,thcv,cbd,cbda,cbdv,cbn,cbg,cbgm,cbgv,cbc,cbcv,cbv,cbt,cbl
0,0,100 OG,Focused,Depression,Citrus,NaN,Limonene,127,0.0,0.0,16,0.0,0.0,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,707 Headband,nan,nan,nan,NaN,NaN,127,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,A-10,"DryMouth,Energetic,Happy,Paranoid,Sleepy,Uplifted,Relaxed","Depression,Insomnia,MuscleSpasms,Pain,Stress","Sweet,Citrus",NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Acapulco Gold,"Anxious,Creative,DryMouth,Euphoric,Paranoid,Relaxed,Uplifted,Happy","LackofAppetite,Pain,Stress,Depression","Citrus,Earthy",NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Afghani Bullrider,"DryMouth,Euphoric,Happy,Paranoid,Relaxed,Uplifted","Depression,Insomnia,Pain,Stress","Earthy,Pine,Sweet",NaN,NaN,127,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
col_1[0][:1]

'e'

In [ ]:
#newlists
effects = []
ailments = []
flavors = []

#create new df for the dummies
df_dummies = pd.DataFrame() 

#get_dummies and concate the new dummy columns and add prefix
for column in col_1:

  df_features = pd.concat([df[column].str.get_dummies(sep=',').add_prefix(column[:1]+"_")], axis=1)
  df = pd.concat([df, df_features], axis=1) #add to original df
  df_dummies = pd.concat([df_dummies, df_features], axis =1)  #create new df for the new columns 

#drop the nan columns 
df = df.drop(['a_nan','e_nan','f_nan'], axis = 1) 
df_dummies = df_dummies.drop(['a_nan','e_nan','f_nan'], axis = 1) 

#append effects, ailments and flavors to lists
for column in df.columns:
  if column.startswith("e"):
    effects.append(column)
  elif column.startswith("a"):
    ailments.append(column)
  elif column.startswith("f"):
    flavors.append(column)

#remove the prefixes from the lists
effects = [i[2:] for i in effects]
ailments = [i[2:] for i in ailments]
flavors = [i[2:] for i in flavors]

#create new lists without the nan column 
Effects = sorted(set(effects))[:-2]
eff_num = len(Effects)
Ailments = sorted(set(ailments))[:-2]
ail_num = len(Ailments)
Flavors = sorted(set(flavors))[:-2]
fla_num = len(Flavors)

#summary
print ("There are {} unique effects listed for these strains.\nThese are: \n{}".format(eff_num, Effects))
print ("There are {} unique ailments listed for these strains.\nThese are: \n{}".format(ail_num, Ailments))
print ("There are {} unique flavors listed for these strains.\nThese are: \n{}".format(fla_num, Flavors))

There are 14 unique effects listed for these strains.
These are: 
['Anxious', 'Creative', 'DryMouth', 'Energetic', 'Euphoric', 'Focused', 'Happy', 'Horny', 'Hungry', 'Paranoid', 'Relaxed', 'Sleepy', 'Talkative', 'Tingly']
There are 8 unique ailments listed for these strains.
These are: 
['Depression', 'Inflammation', 'Insomnia', 'LackofAppetite', 'MuscleSpasms', 'Nausea', 'Pain', 'Seizures']
There are 22 unique flavors listed for these strains.
These are: 
['Ammonia', 'Apple', 'Berry', 'Blueberry', 'Citrus', 'Coffee', 'Earthy', 'Grape', 'Honey', 'Lavender', 'Lemon', 'Lime', 'Mango', 'Mint', 'Orange', 'Pepper', 'Pine', 'Pineapple', 'Rose', 'Skunk', 'Strawberry', 'Sweet']


In [ ]:
df.head(2)
#len(df)

,Unnamed: 0,name,effects,ailment,flavor,location,terpenes,thc,thca,thcv,cbd,cbda,cbdv,cbn,cbg,cbgm,cbgv,cbc,cbcv,cbv,cbt,cbl,e_Anxious,e_Creative,e_DryMouth,e_Energetic,e_Euphoric,e_Focused,e_Happy,e_Horny,e_Hungry,e_Paranoid,e_Relaxed,e_Sleepy,e_Talkative,e_Tingly,e_Uplifted,a_Depression,a_Inflammation,a_Insomnia,a_LackofAppetite,a_MuscleSpasms,a_Nausea,a_Pain,a_Seizures,a_Stress,f_Ammonia,f_Apple,f_Berry,f_Blueberry,f_Citrus,f_Coffee,f_Earthy,f_Grape,f_Honey,f_Lavender,f_Lemon,f_Lime,f_Mango,f_Mint,f_Orange,f_Pepper,f_Pine,f_Pineapple,f_Rose,f_Skunk,f_Strawberry,f_Sweet,f_Vanilla
0,0,100 OG,Focused,Depression,Citrus,NaN,Limonene,127,0.0,0.0,16,0.0,0.0,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,707 Headband,nan,nan,nan,NaN,NaN,127,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_dummies.head(5)
#len(df_dummies)

,e_Anxious,e_Creative,e_DryMouth,e_Energetic,e_Euphoric,e_Focused,e_Happy,e_Horny,e_Hungry,e_Paranoid,e_Relaxed,e_Sleepy,e_Talkative,e_Tingly,e_Uplifted,a_Depression,a_Inflammation,a_Insomnia,a_LackofAppetite,a_MuscleSpasms,a_Nausea,a_Pain,a_Seizures,a_Stress,f_Ammonia,f_Apple,f_Berry,f_Blueberry,f_Citrus,f_Coffee,f_Earthy,f_Grape,f_Honey,f_Lavender,f_Lemon,f_Lime,f_Mango,f_Mint,f_Orange,f_Pepper,f_Pine,f_Pineapple,f_Rose,f_Skunk,f_Strawberry,f_Sweet,f_Vanilla
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,1,1,0,0,1,0,0,1,1,1,0,0,1,1,0,1,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,1,1,1,0,1,0,1,0,0,1,1,0,0,0,1,1,0,0,1,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,1,0,1,0,0,1,1,0,0,0,1,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


ADD THC 

In [ ]:
#create copy of df_dummies
dfd = df_dummies.copy()
dfd.head(5)

,e_Anxious,e_Creative,e_DryMouth,e_Energetic,e_Euphoric,e_Focused,e_Happy,e_Horny,e_Hungry,e_Paranoid,e_Relaxed,e_Sleepy,e_Talkative,e_Tingly,e_Uplifted,a_Depression,a_Inflammation,a_Insomnia,a_LackofAppetite,a_MuscleSpasms,a_Nausea,a_Pain,a_Seizures,a_Stress,f_Ammonia,f_Apple,f_Berry,f_Blueberry,f_Citrus,f_Coffee,f_Earthy,f_Grape,f_Honey,f_Lavender,f_Lemon,f_Lime,f_Mango,f_Mint,f_Orange,f_Pepper,f_Pine,f_Pineapple,f_Rose,f_Skunk,f_Strawberry,f_Sweet,f_Vanilla
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,1,1,0,0,1,0,0,1,1,1,0,0,1,1,0,1,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,1,1,1,0,1,0,1,0,0,1,1,0,0,0,1,1,0,0,1,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,1,0,1,0,0,1,1,0,0,0,1,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


In [ ]:
#columns to be added
columns = ["thc", "cbd"]

for column in columns:
  dfd[column]= df[column]

dfd.head(5)

,e_Anxious,e_Creative,e_DryMouth,e_Energetic,e_Euphoric,e_Focused,e_Happy,e_Horny,e_Hungry,e_Paranoid,e_Relaxed,e_Sleepy,e_Talkative,e_Tingly,e_Uplifted,a_Depression,a_Inflammation,a_Insomnia,a_LackofAppetite,a_MuscleSpasms,a_Nausea,a_Pain,a_Seizures,a_Stress,f_Ammonia,f_Apple,f_Berry,f_Blueberry,f_Citrus,f_Coffee,f_Earthy,f_Grape,f_Honey,f_Lavender,f_Lemon,f_Lime,f_Mango,f_Mint,f_Orange,f_Pepper,f_Pine,f_Pineapple,f_Rose,f_Skunk,f_Strawberry,f_Sweet,f_Vanilla,thc,cbd
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,127,16
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,127,0
2,0,0,1,1,0,0,1,0,0,1,1,1,0,0,1,1,0,1,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,1,1,1,0,1,0,1,0,0,1,1,0,0,0,1,1,0,0,1,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,1,0,1,0,0,1,1,0,0,0,1,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,127,0


In [ ]:
from sklearn.preprocessing import MinMaxScaler  #scaling is important when dealing with discrete data in order to rescale variables to a similar numeric range.

min_max_scaler = MinMaxScaler()
dfd2 = min_max_scaler.fit_transform(dfd)
np.round(dfd2,2)

array([[0.  , 0.  , 0.  , ..., 0.  , 0.02, 0.01],
       [0.  , 0.  , 0.  , ..., 0.  , 0.02, 0.  ],
       [0.  , 0.  , 1.  , ..., 0.  , 0.  , 0.  ],
       ...,
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ]])

In [ ]:
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(n_neighbors=10, algorithm='brute', metric = 'cosine').fit(dfd2) #also try algorithm = 'ball_tree'
distances, indices = model.kneighbors(dfd2)

**BASED ON METHOD**

In [ ]:
print("1. Based on your favourite strain\n2. Based on effects & ailments\n3. Based on your next activity ")
method = int(input("How do you want to choose your strain? "))
print("\n")

if method == 1:
  name_entry = input("what is your favourite weed, based on effects, ailments and flavor? ")

  all_names = list(df.name.values)
  name_entry_indx = all_names.index(name_entry)

  print("\n")
  print("Your choices are: ")
  print("\n")

  for i in indices[name_entry_indx][:]:
    nm = df["name"].iloc[i]
    print(nm)

elif method ==2:
  #TWO POSITIVE IMPACTS
  pos_impacts = ['Creative','Energetic','Euphoric','Focused','Happy','Horny','Relaxed','Sleepy', 'Talkative','Tingly']
  
  print ("1.Creative\n2.Energetic\n3.Euphoric\n4.Focused\n5.Happy\n6.Horny\n7.Relaxed\n8.Sleepy\n9.Talkative\n10.Tingly")
  effect_1, effect_2 = input("Please choose 2 positive effects with a space between: ").split(' ') #enter effect choices
  eff_choice_1 = pos_impacts[int(effect_1) - 1]
  eff_choice_2 = pos_impacts[int(effect_2) - 1]
  
  print (effect_1, effect_2, eff_choice_1, eff_choice_2)
  first_comb = r"(?=.*" + eff_choice_1 + ")(?=.*" + eff_choice_2 + ")"
  new_eff1 = df[df['effects'].str.contains(first_comb, regex =True, na=False)]

  print("\n")
#'----------------------------------------------------------------------------------------------------------------------'
  
  #TWO NEGATIVE IMPACTS
  neg_impacts = ['Anxious','DryMouth','Hungry','Paranoid']
  #4
  print ("1.Anxious\n2.DryMouth\n3.Hungry\n4.Paranoid\n")
  neffect_1, neffect_2 = input("Please choose 2 negative effects you CAN'T live with: ").split(' ') #enter effect choices

  neff_choice_1 = neg_impacts[int(neffect_1) - 1]
  neff_choice_2 = neg_impacts[int(neffect_2) - 1]

  print (neffect_1, neffect_2, neff_choice_1, neff_choice_2)

  second_comb = r"(?=.*" + neff_choice_1 + ")(?=.*" + neff_choice_2 + ")"

  new_eff = new_eff1[~new_eff1['effects'].str.contains(second_comb, regex=True , na=False)]

  print("\n")
#'----------------------------------------------------------------------------------------------------------------------'
  yesno = input ("Are you dealing with an ailment (yes or no)? ")

  #ONE AILMENT
  #out of 8 ailments 
  if yesno =="yes":
  
    ailments = ['Depression', 'Inflammation', 'Insomnia', 'LackofAppetite', 'MuscleSpasms', 'Nausea', 'Pain', 'Seizures']

    print ("1.Depression\n2.Inflammation\n3.Insomnia\n4.LackofAppetite\n5.MuscleSpasms\n6.Nausea\n7.Pain\n8.Seizures")
    ail_1 = input("Please choose one ailment you are dealing with: ") #enter ailment choices

    ail_1_choice = ailments[int(ail_1)-1]

    new_ail = new_eff[new_eff['ailment'].str.contains(ail_1_choice, regex=True,na=False)]

    #print(ail_1_choice)
    #new_ail.head(5)
    print("\n")
    #Get the name of the first choice, the its index 
    choice  = new_ail['name'].iloc[0]
    print("choice is: ",choice)
    all_names = list(df.name.values)
    indx_ch = all_names.index(choice)
    print("index is: ",indx_ch)

    print("\n")
    print("Your choices are: ")
    print("\n")

    for i in indices[indx_ch][:]:
      nm = df["name"].iloc[i]
      print(nm)
#'----------------------------------------------------------------------------------------------------------------------'
  #NO AILMENT
  if yesno =="no":
    #print(second_comb)
    #print (new_eff.head(3))
    print("\n")
    #Get the name of the first choice, the its index 
    choice  = new_eff['name'].iloc[0]
    print("choice is: ",choice)
    all_names = list(df.name.values)
    indx_ch = all_names.index(choice)
    print("index is: ",indx_ch)

    print("\n")
    print("Your choices are: ")
    print("\n")

    for i in indices[indx_ch][:]:
      nm = df["name"].iloc[i]
      print(nm)
#'----------------------------------------------------------------------------------------------------------------------'

elif method ==3:
  print("1. Study or Work\n2. Go on a hot date\n3. Relax/Sleep\n4. Hang out\n5. Travel\n6. Create/Paint/Write")
  activity = int(input("Please choose what you're about to do? "))
  print("\n")

  Study_Work_pos= ['Focused','Energetic']
  Hot_date_pos = ['Horny','Talkative', 'Happy', 'Euphoric']
  Sleep_relax_pos = ['Sleepy', 'Relaxed']
  Hanging_out_pos = ["Happy", "Energetic" "Talkative"]
  Travelling_pos = ["Focused", "Relaxed", "Happy"]
  Creating_Painting_Writing_pos = ["Creative", "Euphoric", "Energetic", "Focused"]
  
  Study_Work_neg= ['Paranoid']
  Hot_date_neg = ["Anxious", "Paranoid"]
  Sleep_relax_neg = ['Anxious']
  Hanging_out_neg = ["DryMouth"]
  Travelling_neg = ["Anxious"]
  Creating_Painting_Writing_neg = ["DryMouth"]

  if activity == 1:
    act_eff1 = df[df['effects'].str.contains(r'(?=.*Focused)(?=.*Energetic)',regex=True, na=False)]
    act_eff = act_eff1[~act_eff1['effects'].str.contains("Paranoid", regex=True , na=False)]
    print ("You want a strain that makes you feel {} but not {}.".format(Study_Work_pos[:], Study_Work_neg[:]))
  elif activity == 2:
    act_eff1 = df[df['effects'].str.contains(r'(?=.*Horny)(?=.*Talkative)(?=.*Happy)(?=.*Euphoric)',regex=True, na=False)]
    act_eff = act_eff1[~act_eff1['effects'].str.contains(r'(?=.*Anxious)(?=.*Paranoid)', regex=True , na=False)]
    print ("You want a strain that makes you feel {} but not {}.".format(Hot_date_pos[:], Hot_date_neg[:]))
  elif activity == 3:
    act_eff1 = df[df['effects'].str.contains(r'(?=.*Sleepy)(?=.*Relaxed)',regex=True, na=False)]
    act_eff = act_eff1[~act_eff1['effects'].str.contains(r'(?=.*Anxious)', regex=True , na=False)]
    print ("You want a strain that makes you feel {} but not {}.".format(Sleep_relax_pos[:], Sleep_relax_neg[:]))
  elif activity == 4:
    act_eff1 = df[df['effects'].str.contains(r'(?=.*Energetic)(?=.*Talkative)(?=.*Happy)',regex=True, na=False)]
    act_eff = act_eff1[~act_eff1['effects'].str.contains("Anxious", regex=True , na=False)]
    print ("You want a strain that makes you feel {} but not {}.".format(Hanging_out_pos[:], Hanging_out_neg[:]))
  elif activity == 5:
    act_eff1 = df[df['effects'].str.contains(r'(?=.*Focused)(?=.*Relaxed)(?=.*Happy)',regex=True, na=False)]
    act_eff = act_eff1[~act_eff1['effects'].str.contains("Anxious", regex=True , na=False)]
    print ("You want a strain that makes you feel {} but not {}.".format(Travelling_pos[:], Travelling_neg[:])) 
  elif activity == 6:
    act_eff1 = df[df['effects'].str.contains(r'(?=.*Creative)(?=.*Focused)(?=.*Energetic)(?=.*Euphoric)',regex=True, na=False)]
    act_eff = act_eff1[~act_eff1['effects'].str.contains("DryMouth", regex=True , na=False)]
    print ("You want a strain that makes you feel {} but not {}.".format(Creating_Painting_Writing_pos[:], Creating_Painting_Writing_neg[:])) 

  print("\n")
  choice  = act_eff['name'].iloc[0]
  print("choice is: ",choice)
  all_names = list(df.name.values)
  indx_ch = all_names.index(choice)
  print("index is: ",indx_ch)

  print("\n")
  print("Your choices are: ")
  print("\n")
  for i in indices[indx_ch][:]:
    nm = df["name"].iloc[i]
    print(nm)


1. Based on your favourite strain
2. Based on effects & ailments
3. Based on your next activity 
How do you want to choose your strain? 3


1. Study or Work
2. Go on a hot date
3. Relax/Sleep
4. Hang out
5. Travel
6. Create/Paint/Write
Please choose what you're about to do? 5


You want a strain that makes you feel ['Focused', 'Relaxed', 'Happy'] but not ['Anxious'].


choice is:  Hempstar
index is:  175


Your choices are: 


Hempstar
Stephen Hawking Kush
Satellite OG
Sage N Sour
Lavender Haze
Obama Kush
Death Star OG
Swazi Gold
Santa Maria
Rollex OG Kush


In [ ]:
df.loc[df['name'] == 'Hempstar']

,Unnamed: 0,name,effects,ailment,flavor,location,terpenes,thc,thca,thcv,cbd,cbda,cbdv,cbn,cbg,cbgm,cbgv,cbc,cbcv,cbv,cbt,cbl,e_Anxious,e_Creative,e_DryMouth,e_Energetic,e_Euphoric,e_Focused,e_Happy,e_Horny,e_Hungry,e_Paranoid,e_Relaxed,e_Sleepy,e_Talkative,e_Tingly,e_Uplifted,a_Depression,a_Inflammation,a_Insomnia,a_LackofAppetite,a_MuscleSpasms,a_Nausea,a_Pain,a_Seizures,a_Stress,f_Ammonia,f_Apple,f_Berry,f_Blueberry,f_Citrus,f_Coffee,f_Earthy,f_Grape,f_Honey,f_Lavender,f_Lemon,f_Lime,f_Mango,f_Mint,f_Orange,f_Pepper,f_Pine,f_Pineapple,f_Rose,f_Skunk,f_Strawberry,f_Sweet,f_Vanilla
175,175,Hempstar,"DryMouth,Euphoric,Focused,Paranoid,Relaxed,Uplifted,Happy","Depression,LackofAppetite,Nausea,Pain,Stress",Earthy,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,1,1,1,0,0,1,1,0,0,0,1,1,0,0,1,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**EXTRANEOUS**